In [ ]:
# for file in mask_*_0.png; do
#   num=$(echo "$file" | grep -oP 'mask_\K\d+')
#   newname=$(printf "mask_%06d.png" "$num")
#   mv "$file" "$newname"
# done


In [ ]:
import numpy as np
import open3d as o3d
import os
from scipy.spatial.transform import Rotation
import copy

voxel_size = 0.005

def load_intrinsics(intrinsics_path):
    """Load camera intrinsics from numpy file"""
    intrinsics = np.load(intrinsics_path)
    return o3d.camera.PinholeCameraIntrinsic(
        width=640, height=480,
        fx=intrinsics[0,0], fy=intrinsics[1,1],
        cx=intrinsics[0,2], cy=intrinsics[1,2]
    )

def create_point_cloud(depth_path, mask_path, intrinsic):
    """Create point cloud from depth image and mask with early downsampling"""
    # Load depth and mask
    depth = np.load(depth_path)
    mask = o3d.io.read_image(mask_path)
    mask_array = np.asarray(mask)

    # Ensure depth and mask have the same shape
    depth_array = np.asarray(depth)
    if depth_array.shape != mask_array.shape:
        raise ValueError(f"Depth and mask shapes do not match: {depth_array.shape} vs {mask_array.shape}")

    print("Depth shape:", depth_array.shape)
    print("Mask shape:", mask_array.shape)

    # Apply mask early to reduce points before creating RGBD image
    depth_array = depth_array * (mask_array > 0)  # Zero out non-masked areas

    # Create RGBD image with masked data
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        o3d.io.read_image(mask_path.replace('mask', 'rgb')),
        o3d.geometry.Image(depth_array.astype(np.float32)),
        depth_scale=1000.0,
        convert_rgb_to_intensity=False
    )

    # Create initial point cloud
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image,
        intrinsic
    )

    pcd = pcd.voxel_down_sample(voxel_size=voxel_size)

    # Estimate normals
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=50))

    # Remove invalid points
    pcd.remove_non_finite_points()

    print("Points shape after downsampling:", len(pcd.points))

    return pcd



def visualize_result(point_clouds, poses):
    """Visualize all point clouds in given poses"""
    coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
    geoms = [coordinate_frame]

    for i, (pcd, pose) in enumerate(zip(point_clouds, poses)):
        pcd_transformed = copy.deepcopy(pcd)
        pcd_transformed.transform(pose)
        geoms.append(pcd_transformed)

    # Visualize the geometries
    o3d.visualization.draw_geometries(
        geoms,
        window_name="Final Visualization"
    )

def register_point_clouds(point_clouds, initial_poses):
    """Perform multiway registration using pose graph optimization with loop closure"""
    pose_graph = o3d.pipelines.registration.PoseGraph()

    # Add initial pose for the first point cloud
    pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(np.eye(4)))

    # Pairwise registration and adding edges to the pose graph
    for i in range(len(point_clouds) - 1):
        source = point_clouds[i]
        target = point_clouds[i + 1]

        # Perform pairwise registration
        icp_result = o3d.pipelines.registration.registration_icp(
            source, target, max_correspondence_distance=0.02,
            estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane()
        )

        # Add edge to pose graph
        pose_graph.edges.append(o3d.pipelines.registration.PoseGraphEdge(
            i, i + 1, icp_result.transformation, uncertain=False
        ))

        # Add node for the next point cloud
        pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(np.linalg.inv(icp_result.transformation)))

    # Add loop closure edges
    for i in range(len(point_clouds)):
        for j in range(i + 2, len(point_clouds)):
            source = point_clouds[i]
            target = point_clouds[j]

            # Perform pairwise registration for loop closure
            icp_result = o3d.pipelines.registration.registration_icp(
                source, target, max_correspondence_distance=0.02,
                estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane()
            )

            # Add loop closure edge to pose graph
            pose_graph.edges.append(o3d.pipelines.registration.PoseGraphEdge(
                i, j, icp_result.transformation, uncertain=True
            ))

    # Optimize the pose graph
    option = o3d.pipelines.registration.GlobalOptimizationOption(
        max_correspondence_distance=voxel_size * 15,
        edge_prune_threshold=0.25,
        reference_node=0
    )
    o3d.pipelines.registration.global_optimization(
        pose_graph,
        o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt(),
        o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria(),
        option
    )

    # Extract optimized poses
    optimized_poses = [node.pose for node in pose_graph.nodes]

    return optimized_poses

def main():
    base_dir = "/home/yilong/ObAct_realworld/tasks/mug"
    intrinsics_path = "/home/yilong/ObAct_realworld/config/d405_intrinsic_right.npy"
    
    # Load camera intrinsics
    intrinsic = load_intrinsics(intrinsics_path)
    
    # Load all point clouds
    point_clouds = []
    for i in range(3, 8):  # Frames 3 to 8
        frame_str = f"{i}"
        depth_path = os.path.join(base_dir, f"depth_{frame_str}.npy")
        mask_path = os.path.join(base_dir, f"mask_{frame_str}.png")

        print(f"Loading frame {i}: depth from {depth_path}, mask from {mask_path}")

        try:
            pcd = create_point_cloud(depth_path, mask_path, intrinsic)
            point_clouds.append(pcd)
        except Exception as e:
            print(f"Error processing frame {i}: {str(e)}")
            continue

    if not point_clouds:
        print("No point clouds could be loaded")
        return

    # Initialize initial poses as identity matrices
    initial_poses = [np.eye(4) for _ in point_clouds]

    # Register point clouds
    optimized_poses = register_point_clouds(point_clouds, initial_poses)

    if not optimized_poses:
        print("Registration failed")
        return

    # Visualize result
    visualize_result(point_clouds, optimized_poses)

if __name__ == "__main__":
    main()

Loading frame 3: depth from /home/yilong/ObAct_realworld/tasks/mug/depth_3.npy, mask from /home/yilong/ObAct_realworld/tasks/mug/mask_3.png
Depth shape: (480, 640)
Mask shape: (480, 640)
Points shape after downsampling: 608
Loading frame 4: depth from /home/yilong/ObAct_realworld/tasks/mug/depth_4.npy, mask from /home/yilong/ObAct_realworld/tasks/mug/mask_4.png
Depth shape: (480, 640)
Mask shape: (480, 640)
Points shape after downsampling: 584
Loading frame 5: depth from /home/yilong/ObAct_realworld/tasks/mug/depth_5.npy, mask from /home/yilong/ObAct_realworld/tasks/mug/mask_5.png
Depth shape: (480, 640)
Mask shape: (480, 640)
Points shape after downsampling: 731
Loading frame 6: depth from /home/yilong/ObAct_realworld/tasks/mug/depth_6.npy, mask from /home/yilong/ObAct_realworld/tasks/mug/mask_6.png
Depth shape: (480, 640)
Mask shape: (480, 640)
Points shape after downsampling: 735
Loading frame 7: depth from /home/yilong/ObAct_realworld/tasks/mug/depth_7.npy, mask from /home/yilong/O

In [4]:
depth = np.load("/home/yilong/ObAct_realworld/tasks/mug/depth_000001.npy")
mask = o3d.io.read_image("/home/yilong/ObAct_realworld/tasks/mug/mask_000001.png")
print("Depth shape:", depth.shape)
print("Mask shape:", np.asarray(mask).shape)

Depth shape: (480, 640)
Mask shape: (480, 640)
